In [7]:
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import branca
pd.set_option("display.max_rows", None)



## Task One: summarizing number of solar panels and Jul-Jan GHI difference for MA


In [8]:
data = pd.read_csv('https://raw.githubusercontent.com/bimewok/geospatial_exercise/main/outputs/zip_stats.csv', index_col=0)


display(data)

,zip_code,num_solar_panels,jul_jan_ghi_diff
0,1001,146.0,178.200264
1,1002,397.0,177.233295
2,1003,23.0,178.953629
3,1005,10.0,177.658722
4,1007,231.0,176.342252
5,1008,1.0,178.175676
6,1009,0.0,175.875531
7,1010,15.0,178.214529
8,1011,1.0,177.085038
9,1012,1.0,174.907230


## Task Two: Find areas where January GHI is above the MA average

In [11]:
import pandas as pd


data = gpd.read_file('https://raw.githubusercontent.com/bimewok/geospatial_exercise/main/outputs/high_jan_ghi.geojson') 
data['idx'] = np.arange(len(data))
data['GHI'] = [round(i, 1) for i in data['ghi']]
min_ghi = np.min(data['GHI'])
max_ghi = np.max(data['GHI'])
                     
colorscale = branca.colormap.linear.YlOrRd_09.scale(min_ghi, max_ghi)
ghi_series = data.set_index("idx")["GHI"]


def style_function(feature):
    ghi = ghi_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.9,
        "weight": 0,
        "fillColor": "#black" if ghi is None else colorscale(ghi),
    }


m = folium.Map([42.4, -71], zoom_start=8, tiles='Stamen Toner')
h = folium.GeoJson(data,
              style_function=style_function).add_to(m)
h.add_child(
    folium.features.GeoJsonTooltip(['GHI'], labels=False)
)

m 

## Task Three: Make a surface model from lidar data

In [16]:

import os
import folium

m = folium.Map([42.3782, -71.06309], zoom_start=14, tiles="OpenStreetMap")

img = folium.raster_layers.ImageOverlay(
    name="Mercator projection SW",
    image='https://github.com/bimewok/geospatial_exercise/blob/main/intermediate_data/hs.jpg?raw=true',
    bounds=[[42.369275, -71.080845], [42.390549, -71.047266]],
    opacity=1,
    interactive=True,
    cross_origin=False,
    zindex=1,
)

folium.Popup("I am an image").add_to(img)

img.add_to(m)
folium.LayerControl().add_to(m)
m


## Task Four: Use surface model to extract heights of buildings

In [ ]:
import pandas as pd
import branca

One = gpd.read_file(r'C:\garrett_workspace\PROJECTS\github\geospatial_exercise\outputs\charlestown_buildings_heights.shp') 
One['idx'] = np.arange(len(One))
One['height '] = [round(i, 1) for i in One['height']]
min_ghi = np.min(One['height '])
max_ghi = np.max(One['height '])
colorscale = branca.colormap.linear.YlOrRd_09.scale(min_ghi, max_ghi)
employed_series = One.set_index("idx")["height "]


def style_function(feature):
    employed = employed_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.9,
        "weight": 0,
        "fillColor": "#black" if employed is None else colorscale(employed),
    }


mapa = folium.Map([42.3782, -71.06309], zoom_start=14, tiles="OpenStreetMap")
h = folium.GeoJson(One,
              style_function=style_function).add_to(mapa)
h.add_child(
    folium.features.GeoJsonTooltip(['height '], labels=True)
)

mapa 